# Notebook para el procesamiento de los datos extraídos

Notebook donde se generan nuevos conjuntos de datos con ajustes necesarios para su posterior procesado, a partir de los datos extraidos en el notebook 01_data_extraction.ipynb 

In [ ]:
import pandas as pd
import ast
import pycountry_convert as pc

De manera general, comentar que se hace principalmente eliminación de columnas que no se van a utilizar así como renombrar otras. También se ajusta el tipo de dato para ciertas columnas para facilitar las operaciones.

#### Resultados
Unión del conjunto de datos de los resultados de carreras y de los sprints a partir de los CSVs generados. El nuevo conjunto de datos se exporta en formato CSV

##### Resultados carreras

In [ ]:
# Leer CSV original
raw_race_results_df = pd.read_csv('../data/results_2000-2024.csv')

# Convertir columna Results a un tipo de dato anidado en Python
raw_race_results_df['Results'] = raw_race_results_df['Results'].apply(ast.literal_eval)
rows = []
results_expanded = []
# Expandir columna Results para tener una fila por piloto a partir de los datos de Results
for index, row in raw_race_results_df.iterrows():
    results_list = row['Results']
    for result in results_list:
        rows.append(row.drop('Results'))
        results_expanded.append(result)

expanded_rows_df = pd.DataFrame(rows)
results_normalized_df = pd.json_normalize(results_expanded) # Aplanar estructuras anidadas en el dataframe
# Concatenar los resultados con el resto de datos
race_results_df = pd.concat([expanded_rows_df.reset_index(drop=True), results_normalized_df.reset_index(drop=True)], axis=1)
race_results_df['points'] = pd.to_numeric(race_results_df['points'], errors='coerce') # Convertir a integer

# Convertir columna Circuit a un tipo de dato anidado en Python
race_results_df['Circuit'] = race_results_df['Circuit'].apply(ast.literal_eval)
circuits_normalized = pd.json_normalize(race_results_df['Circuit']) # Aplanar estructuras anidadas en el dataframe

# Concatenar los circuitos con el resto de datos
race_results_df = pd.concat([race_results_df.drop(columns=['Circuit']), circuits_normalized],axis=1)

# Borrar columnas innecesarias
race_results_df.drop(columns=['url', 'Location.lat', 'Location.long', 'Location.locality', 'Location.country', 'Driver.url', 'Driver.permanentNumber', 'Constructor.url', 'time'], axis=1, inplace=True)

# Renombrar columnas
race_results_df.rename(columns={'Driver.driverId': 'driverId', 'Driver.code': 'driverCode', 'Driver.givenName': 'driverName', 'Driver.familyName': 'driverSurname', 'Driver.dateOfBirth': 'driverBirth',
'Driver.nationality':'driverNationality', 'Constructor.constructorId': 'constructorId', 'Constructor.name': 'constructorName', 'Constructor.nationality': 'constructorNationality', 'Time.millis': 'timeMillis', 'Time.time': 'time', 'FastestLap.rank': 'fastestLapRank',
'FastestLap.lap': 'fastestLap', 'FastestLap.Time.time': 'fastestLapTime', 'FastestLap.AverageSpeed.speed': 'fastestLapAverageSpeed', 'FastestLap.AverageSpeed.units': 'fastestLapAverageSpeedUnit'},  inplace=True)

# Convertir tipos de datos
race_results_df = race_results_df.astype({'position': 'int64', 'points': 'int64', 'grid': 'int64', 'laps': 'int64'})

race_results_df.head()


##### Resultados sprint races

In [ ]:
# Leer CSV original
raw_sprint_results_df = pd.read_csv('../data/sprint_results_2000-2024.csv')

# Convertir columna Results a un tipo de dato anidado en Python
raw_sprint_results_df['SprintResults'] = raw_sprint_results_df['SprintResults'].apply(ast.literal_eval)

# Expandir columna SprintResults para tener una fila por piloto a partir de los datos de SprintResults
rows = []
results_expanded = []
for index, row in raw_sprint_results_df.iterrows():
    results_list = row['SprintResults']
    for result in results_list:
        rows.append(row.drop('SprintResults'))
        results_expanded.append(result)

expanded_rows_df = pd.DataFrame(rows)
results_normalized_df = pd.json_normalize(results_expanded) # Aplanar estructuras anidadas en el dataframe

# Concatenar los resultados de las carreras sprint con el resto de datos
sprint_results_df = pd.concat([expanded_rows_df.reset_index(drop=True), results_normalized_df.reset_index(drop=True)], axis=1)

sprint_results_df['points'] = pd.to_numeric(sprint_results_df['points'], errors='coerce') # Convertir a integer

# Convertir columna Circuit a un tipo de dato anidado en Python
sprint_results_df['Circuit'] = sprint_results_df['Circuit'].apply(ast.literal_eval) 
circuits_normalized = pd.json_normalize(sprint_results_df['Circuit']) # Aplanar estructuras anidadas en el dataframe

# Concatenar los circuitos con el resto de datos
sprint_results_df = pd.concat([sprint_results_df.drop(columns=['Circuit']), circuits_normalized],axis=1)

# Borrar columnas innecesarias
sprint_results_df.drop(columns=['url', 'Location.lat',	'Location.long', 'Location.locality', 'Location.country', 'Driver.url', 'Driver.permanentNumber', 'Constructor.url', 'time'], axis=1, inplace=True)

# Renombrar columnas
sprint_results_df.rename(columns={'Driver.driverId': 'driverId', 'Driver.code': 'driverCode', 'Driver.givenName': 'driverName', 'Driver.familyName': 'driverSurname', 'Driver.dateOfBirth': 'driverBirth',
'Driver.nationality':'driverNationality', 'Constructor.constructorId': 'constructorId', 'Constructor.name': 'constructorName', 'Constructor.nationality': 'constructorNationality', 'Time.millis': 'timeMillis', 'Time.time': 'time', 'FastestLap.rank': 'fastestLapRank',
'FastestLap.lap': 'fastestLap', 'FastestLap.Time.time': 'fastestLapTime', 'FastestLap.AverageSpeed.speed': 'fastestLapAverageSpeed', 'FastestLap.AverageSpeed.units': 'fastestLapAverageSpeedUnit', 'position':'sprintPosition', 'points':'sprintPoints', 'grid':'sprintGrid', 'laps':'sprintLaps'},  inplace=True)

# Convertir
sprint_results_df = sprint_results_df.astype({'sprintPosition': 'int64', 'sprintPoints': 'int64', 'sprintGrid': 'int64', 'sprintLaps': 'int64'})

sprint_results_df.head()


##### Unión de ambos conjuntos de datos

In [ ]:
# Unir los resultados de las carreras sprint con los rows de los resultados de las carreras
sprint_results_df['sprintRace']= True
sprint_results_df.head()
all_results_df = pd.merge(race_results_df, sprint_results_df[['circuitId', 'season', 'driverId','sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints','sprintRace']], on=['circuitId', 'season', 'driverId'], how='left')

# Rellenar NaN en la columna sprintRace con False, indicando que no hubo sprint race para el Gran Premio si era NaN 
all_results_df['sprintRace'] = all_results_df['sprintRace'].fillna(False)

# Convertir valores
all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']] = all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']].apply(pd.to_numeric, errors='coerce')

# Valores a columnas NaN
all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']] = all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']].fillna(0)
all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']] = all_results_df[['fastestLapRank', 'fastestLap', 'sprintPosition', 'sprintGrid', 'sprintLaps', 'sprintPoints']].astype(int)

# Nueva columna con la suma de los puntos para un fin de semana: Race points + Sprint points
all_results_df['weekendPoints'] = all_results_df['points'] + all_results_df['sprintPoints']

# Crear nuevo CSV
all_results_df.to_csv('../data/race_and_sprint_results_2000-2024.csv', index=False)

Cálculo de los puntos acumulados por piloto a lo largo de la temporada

In [ ]:
all_results_df['cumulative_points'] = all_results_df.groupby(['season','driverId'])['weekendPoints'].cumsum()
all_results_df.tail(25)


#### Circuitos por temporada

Función para obtener el continente al que pertenece cada país donde se celebra gran premio para añadirlo al conjunto de datos

In [ ]:
def get_continent_name(country_name):
    country_code = pc.country_name_to_country_alpha2(country_name)
    continent_code = pc.country_alpha2_to_continent_code(country_code)
    continent_dict = {
        "NA": "North America",
        "SA": "South America",
        "AS": "Asia",
        "AF": "Africa",
        "OC": "Oceania",
        "EU": "Europe",
        "AQ" : "Antarctica"
    }
    return continent_dict[continent_code]


Limpieza del conjunto de datos de circuitos

In [ ]:
# Leer CSV de circuitos
df_seasons_circuits = pd.read_csv('../data/seasons_circuits_2000-2024.csv')

# Eliminar columnas innecesarias
df_dropped_circuits = df_seasons_circuits.drop(columns=['time', 'ThirdPractice', 'FirstPractice', 'SecondPractice', 'url', 'Sprint', 'Qualifying'])

# Convertir a una estructura anidada en Python
df_dropped_circuits['Circuit'] = df_dropped_circuits['Circuit'].apply(ast.literal_eval)

# Aplanar circuitos y concatenarlo al dataframe original, eliminando la columna con la estructura anidada
df_flat_circuits = pd.concat(
    [df_dropped_circuits.drop(columns=['Circuit']), pd.json_normalize(df_dropped_circuits['Circuit'])],
    axis=1
)

# Renombrar columnas
df_flat_circuits.rename(columns={'Location.lat':'latitude', 'Location.long':'longitude', 'Location.locality':'locality', 'Location.country':'country'}, inplace=True)

# Borrar más columnas
df_flat_circuits.drop(columns=['url'], inplace=True)

# Reemplazar algunos nombres de países para hacerlo compatible con la librería pycountry_convert
df_flat_circuits['country'] = df_flat_circuits['country'].replace({'UK': 'United Kingdom', 'USA': 'United States', 'UAE': 'United Arab Emirates', 'Korea': 'South Korea'})

# Asignar contienente a cada fila
df_flat_circuits['continent'] = df_flat_circuits['country'].apply(get_continent_name)

# Crear nuevo CSV
df_flat_circuits.to_csv('../data/cleaned_circuits_2000-2024.csv', index=False)


#### Pitstops
Limpieza del conjunto de datos de los pitstops

In [ ]:
# Cargar CSV original
raw_pitstops_df = pd.read_csv('../data/raw_pitstops.csv')

# Convertir columna PitStops a una estructura anidada en Python
raw_pitstops_df['PitStops'] = raw_pitstops_df['PitStops'].apply(ast.literal_eval)

# Expandir columna PitStops para tener una fila por piloto a partir de los datos de PitStops
rows = []
pitstops_expanded = []
for index, row in raw_pitstops_df.iterrows():
    pitstop_list = row['PitStops']
    for pitstop_item in pitstop_list:
        rows.append(row.drop('PitStops'))
        pitstops_expanded.append(pitstop_item)

expanded_rows_df = pd.DataFrame(rows)
pitstops_normalized_df = pd.json_normalize(pitstops_expanded) # Aplanar estructuras anidadas en el dataframe

# Aplanar circuitos y concatenarlo al dataframe original, eliminando la columna con la estructura anidada
pitstops_rows_df = pd.concat([expanded_rows_df.reset_index(drop=True), pitstops_normalized_df.reset_index(drop=True)], axis=1)
 
pitstops_rows_df['Circuit'] = pitstops_rows_df['Circuit'].apply(ast.literal_eval) # Aplanar estructuras anidadas en el dataframe
# Extraer el valor de la key circuitId en el diccionario Circuit y añadir circuitId a una columna nueva en el dataframe
pitstops_rows_df['circuitId'] = pitstops_rows_df['Circuit'].apply(lambda x: x.get("circuitId"))

# Eliminar columnas
pitstops_rows_df.drop(columns=['url', 'Circuit','time'], axis=1, inplace=True)

# Añadir nombre de los pilotos al dataframe utilizando los valores presentes en el CSV race_and_sprint_results_2000-2024.csv
results_df = pd.read_csv('../data/race_and_sprint_results_2000-2024.csv')
teams_from_results_df = results_df[['season', 'driverId', 'driverName', 'driverSurname', 'constructorId', 'constructorName']]
teams_from_results_df = teams_from_results_df.groupby(['season', 'driverId']).first().reset_index()
drivers_pitstops_df = pd.merge(pitstops_rows_df, teams_from_results_df[['season','driverId','driverName', 'driverSurname', 'constructorId', 'constructorName']], on=['season','driverId'], how='left')
drivers_pitstops_df['driverFullName'] = drivers_pitstops_df['driverName'] + ' ' + drivers_pitstops_df['driverSurname']

# Crear nuevo CSV
drivers_pitstops_df.to_csv('../data/cleaned_pitstops.csv', index=False)